# Capstone Project: 

## Machine Learning Engineer Nanodegree





Wei-Chuang Chan

In [ ]:
import numpy as np
import pandas as pd
from IPython.display import display

In [ ]:
#Import data for expenditure, and total completion number as performance reference
delta_00=pd.read_csv("Delta_Cost/delta_public_release_00_13.csv")
delta_87=pd.read_csv("Delta_Cost/delta_public_release_87_99.csv")

In [ ]:
columns01=['academicyear','has_completions','totalcompletions',
           'instruction01','research01','rschpub01','acadsupp01','studserv01','pubserv01','instsupp01','acadinststud01',
           'opermain01','depreciation01','grants01','auxiliary01','hospital01','independ01',
           'otheroper01','othernon01','other01']
data_00=delta_00[columns01]
data_87=delta_87[columns01]

In [ ]:
i_data=pd.concat([data_00,data_87])

Compile all data with needed fields from 1987 to 2013.

In [ ]:
#Excluding data point with out report of totalcompletion
i_data=i_data[i_data.has_completions !=0 ]

In [ ]:
#Check the length of data set
print "Length of data set is {}".format(len(i_data))
print "Quantity of missing data in each field:"
#Count the missing number in each column.
print i_data.isnull().sum()

To eliminate fields with too much missing values, if a features contains more than 95% of missing value, removing the field from dataset.

In [ ]:
for feature in i_data.keys():
    if i_data[feature].isnull().sum() >= (len(i_data)*0.95):
        i_data.drop(feature,axis=1,inplace=True)

Remove data points contains missing value.

In [ ]:
i_data.dropna(inplace=True)

In [ ]:
i_data.reset_index(drop=True);

In [ ]:
y=i_data['totalcompletions']
X=i_data.drop(['academicyear','totalcompletions','has_completions'],axis=1)

In [ ]:
from pandas.tools.plotting import table

ax = plt.subplot(111, frame_on=False) # no visible frame
ax.xaxis.set_visible(False)  # hide the x axis
ax.yaxis.set_visible(False)  # hide the y axis

table(ax, X.describe()); # where df is your data frame

plt.savefig('table_all.png',dpi=300,orientation='potrait', bbox_inches='tight',pad_inches=0);
table(ax, X.iloc[:,:7].describe());
plt.savefig('table_7.png',dpi=300,orientation='potrait', bbox_inches='tight',pad_inches=0);
table(ax, X.iloc[:,7:].describe());
plt.savefig('table_8.png',dpi=300,orientation='potrait', bbox_inches='tight',pad_inches=0);

In [ ]:
display(X.iloc[:,:8].describe(),X.iloc[:,8:17].describe())

In [ ]:
np.random.seed(56)
indices=np.random.choice(a=X.index,size=3,replace=False)
sample= pd.DataFrame(X.copy().loc[indices]).reset_index(drop = True)



In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

#Visualize the correlation for each feature pair
%matplotlib inline
corr=X.corr()
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
cmap = sns.diverging_palette(255, 2, as_cmap=True)
sns.set_style("whitegrid")
sns.heatmap(corr, mask=mask,
            cmap=cmap,
            vmax=1,
            square=True,
            linewidths=.2,
           )
plt.title('Correlation between\nexpenditure categories')
plt.tight_layout()
plt.savefig("correlation_chart.png");

Sample 0 - lower than 25th percentile on instruction, student service, institution support, opermain01, depreciation01, grants01, auxiliary01. More than 25th percentile and lower than 50th percentile on public service, acadinststud01, public service, other01. More than 50 percentile while below average research, rschpub01, acadinststud01. The expenditure of sample 0 falls below average on the lower part of the data set. 


Sample 1 - lower than 25th percentile on other01. More than 50th percentile but lower than average on research, rschpub01. More than 75th percentile on instruction, acadsupp01, student service, public service, instsupp01, acadinststud01, opermain01, depreciation01, grants01, auxiliary01. Sample 1 has higher expense on most categories other than other01 and research. Sample 1 may represent more students or enrollments but fewer allocation of budget on research project. 

Sample 2 - lower than 25th percentile on instructions, rschpub01, acadsupp01, studserv01, pubserv01, instsupp01, opermain01, depreciation01, grants01. Higher than 25th percentile and lower than 50th percentile on research, auxiliary, other01. Sample 2 consist lower expenditure on all categories, and the sample may be reported by an institute of a smaller scale.

The data set will be randomly split into training and testing sets and the former will further explored below.

In [ ]:
from sklearn import cross_validation
#Divide label and data and randomly choose training and testing set
X_train, X_test, y_train, y_test=cross_validation.train_test_split(X, y, test_size=0.25, random_state=72)

Identify the outliers in the training set, and remove outlier if significantly impact performance. To avoid deleting relevant data, removal of outlier will be implemented only if performance significantly drops.

Visualize the correlation between features after removal of outliers. Instruction01 appears to have high correlation with other features, and other expenditure has low correlation with any other features. Features including depreciation01 and pubserv01 have no significant correlation with most other feature in general, but pubserv01 has more than 0.5 of correlation with research01 and rschpub01. Features studserv01, grants01, and auxiliary01 have high correlation with around half of the other features but also show low correlation with the other half of the features.

As the plot shows, instruction is highly correlated with nearly all kinds of expenditure, and so does research expense. While other01, pubserv01, and depreciateion01 could be the most distinguishable features among all.


In [ ]:
from sklearn.tree import DecisionTreeRegressor
benchmark_tree=DecisionTreeRegressor();
benchmark_tree.fit(X_train,y_train);

In [ ]:
import math
def aic_bic(model,X,y):
    n = len(X)
    p = len(X.columns)
    RSS = sum((y-model.predict(X))**2)
    AIC = 2*p+n*np.log(RSS/n)
    #n + n*np.log(2*math.pi) + n*np.log(RSS/n) + 2*(p + 1)
    BIC = n*np.log(RSS/n)+p*np.log(n)
    #n + n*np.log(2*math.pi) + n*np.log(RSS/n) + (np.log(n))*(p + 1)
    print "AIC:",AIC
    print "BIC:",BIC
    return AIC, BIC


In [ ]:
def adj_r2_score(score,predictors,datalength):
    adj =1-((1-score)*(datalength-1))/(datalength-predictors-1)
    return adj

In [ ]:
bench_aic, bench_bic=aic_bic(benchmark_tree, X_test, y_test)
print "Adjusted R2 score:",adj_r2_score(benchmark_tree.score(X_test,y_test),len(X_test.columns),len(X_test))

### Algorithms and Techniques

The tasks of the project including finding the expenditure pattern, and compare the performance of prediction between expenditure pattern and original expenditure. Principle Component Analysis(PCA) will be implemented and the componenet will be used to represent expenditure allocation, and expenditure pattern. Prior to PCA transformation, a copy of training data will be scaled before fit and transform via PCA. The dimensions acquired through PCA will be used to represent expenditure patterns in the following supervised learning phase.

In the second phase, 3 models will first be compared by adjusted R-squared score and time cost. The model in the selection pool includes Decistion Tree Regressor, Nearest Neighbor Regressor, and Support Vector Regressor. A k-fold validation will be used to evaluate the performance of model. The model with best estimator will be fit by whole training set and transformed training set, and the performance to predict testing set will be used to determine whether expenditure pattern makes better predictor of completion number.

Transforming data accompanies loss of information while using principle components instead of original expenditure may reduce the dimensions of data set. By comparing the time spent and performance of model trained via different data, the goal of this project is to find the better approach to predict performance of higher education institute.

A randomly selected sample set will be present as a reference.

### Benchmark
The aim is to compare the performance of model on original data and pca transformed data, and the performance score of model trained by original data will be the benchmark comparing with transformed version. If there are null value generated in the transformation, the data point contains the value.

## III. Methodology
### Data Preprocessing

Features are selected for analysis as stated above, among all feature taken. As discovered in data exploration, the data set contains great amount of missing data. The features have more than 95% of missing value will also be removed. To obtain the most complete data from the remaining data, any data contains missing value are removed on the next step. The effect of outliers will be discussed and decided whether to be deleted or not.

Prior to PCA analysis and transformation, the log of data set is taken and standardized. During the tranformation process, any data point found to have null value will be removed.

To properly transform and use testing set, there are several data points that contains infinity value, which cannot be processed by PCA, have to be removed.

### Implementation

After preprocessing, the log of data set will be taken and standardize, and it will go through PCA analysis to retrive components to be used as alternative data. Regressional models will be fitted to transformed data and original data set and compared by their adjusted R2 score and time cost. The best performed model will go through grid search CV in order to be tuned.

### Refinement

DecisionTreeRegressor is chosen as the predictive models for both data set, and tuned for both original training data and transformed data. The final model tuned for original data shows adjusted R2 score of 0.9438, and R2 score of model trained with transformed data is 0.8769. While untuned model has 0.9330 on original data and 0.8677 on transformed data.

In [ ]:
#Examine skewness and kurtosis of data
import scipy.stats
for feature in X_train.keys():
    print "{} - Skewness={}, Kurtosis={}".format(feature, np.round(scipy.stats.skew(X_train[feature]),4),np.round(scipy.stats.kurtosis(X_train[feature]),4))


Normal distribution has 0.03 skewness and 2.96 kurtosis.
The shape of distribution of features are all positively skewed, and the data points tend to sit within a narrow range and resulted high kurtosis. 

In [ ]:
pd.scatter_matrix(X_train, alpha=0.2, figsize=(15,15),diagonal='kde');
plt.savefig('train_scatter_matrix.png',bbox_inches='tight');

In [ ]:

def train_standardize(X,X_train):
    X_stand=(X-X_train.mean())/X_train.std()
    return X_stand
X_train_norm=train_standardize(X_train,X_train)

After standardize log data, it is more clear that sample 0 and sample 2 fall on the lower part of the data set and most of sample 1's expenditure exceed 1 standard deviation over mean. 

The scatter matrix of log data also shows the correlation between each pair of features. Intruction again shows strong correlation with many other features. Other01 doesn't appear to have no notable correlation with others.
#TODO:Elaborate more on scatter matrix.

In [ ]:
display(X_train_norm.head(3).iloc[:,:7],X_train_norm.head(3).iloc[:,7:])

In [ ]:
#Implement Principle Componenet Analysis
from sklearn.decomposition import PCA
pca = PCA()
pca.fit(X_train_norm)
X_train_pca=pca.transform(X_train_norm)
pca_samples=pca.transform(X_train_norm.copy().head(3))
print np.cumsum(pca.explained_variance_)/np.sum(pca.explained_variance_)

In [ ]:
dimensions=["Dimension {}".format(n+1) for n in range(pca.n_components_)]
X_train_pca=pd.DataFrame(X_train_pca, columns=dimensions)
display(X_train_pca.head(3).iloc[:,:7],X_train_pca.head(3).iloc[:,7:])

In [ ]:

# PCA components
components = pd.DataFrame(np.round(pca.components_, 4), columns = X_train_norm.keys())
components.index = dimensions

# PCA explained variance
ratios = pca.explained_variance_ratio_.reshape(pca.n_components_, 1)
variance_ratios = pd.DataFrame(np.round(ratios, 4), columns = ['Explained Variance'])
variance_ratios.index = dimensions

# Create a bar plot visualization
fig, ax = plt.subplots(figsize = (20,7))

# Plot the feature weights as a function of the components
components.plot(ax = ax, kind = 'bar');
ax.set_ylabel("Feature Weights")
ax.set_xticklabels(dimensions)


# Display the explained variance ratios
for i, ev in enumerate(pca.explained_variance_ratio_):
    ax.text(i-0.2, ax.get_ylim()[1], "Explained \n Variance\n  %.4f"%(ev))

# Return a concatenated DataFrame
pca_data=pd.concat([variance_ratios, components], axis = 1)
pca_data;
plt.savefig("PCA.png", bbox_inches='tight')

In [ ]:
X_test_stand=train_standardize(X_test,X_train)
X_test_pca=pca.transform(X_test)
X_test_pca=pd.DataFrame(X_test_pca, columns=dimensions)
tree_pca=DecisionTreeRegressor()
tree_pca.fit(X_train_pca.iloc[:,:10],y_train)
print len(X_train_pca.iloc[:,:10]),len(X_test_pca.iloc[:,:10].columns)
tree_aic, tree_bic=aic_bic(tree_pca, X_test_pca.iloc[:,:10], y_test)
print "Adjusted R2 score:",adj_r2_score(tree_pca.score(X_test_pca.iloc[:,:10],y_test),len(X_test_pca.iloc[:,:10].columns),len(X_test_pca.iloc[:,:10]))


## IV. Results
### Model Evaluation and Validation

The final model was chosen by its adjusted R2 score and will be evaluated throuhg kfold cross validation in the next version. The result of model evaluation requires further improvment. Although the model seems to present quite nice score on predicting testing data. The result also goes against initial assumption that transformed data will be a better predictor. An AIC or BIC comparison between the models might be necesary since the transformation of data accompanies with loss of information.

### Justification
As stated above, the performace of transformed data is a weaker predictor comparing with original data set. PCA does not narrow down the dimensionality and further analysis and feature selection might be necessary. The result is not sufficient to solve the initial question by lack of evaluation of model.

Among all 3 models, Decision Tree Regressor has the highest adjusted R-squared score and grid search will be implemented to obtain best estimator.

In [ ]:
from sklearn.metrics import make_scorer
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import ShuffleSplit
def performance_metric(y_true, y_predict):
    score = r2_score(y_true, y_predict)
    return score

def fit_model(X,y):
    cv_sets= ShuffleSplit(X.shape[0],test_size = 0.25, random_state = 50)
    regressor = tree.DecisionTreeRegressor()
    params= {
             'min_samples_split':[2,4,6], 
             'min_weight_fraction_leaf':[0.0,0.1,0.5]
    }
    scoring_fnc = make_scorer(performance_metric)
    grid = GridSearchCV(regressor,params,scoring=scoring_fnc,cv=cv_sets)
    g = grid.fit(X, y)

    return g.best_estimator_

## V. Conclusion


### Free-Form Visualization
Comparison of the performance can be presented by AUC. 

### Reflection
This project targets to find the principle components of the expenditure of higher education intitutes and used them as a better predictor of performance, which is defined as total number of degree, certificate, and award completion. In the process of the project, it is quite challenging to find a proper benchmark for the model. Also, the evaluation of models is another challenging. 

### Improvement
While predicting the performance of an institute, rather than using expenditure of current year, previous years should be also taken into consideration for any degree, certificate, awards can take longer than an year to be completed.

On the execution part, this project has a great room for improvment by choosing better evaluation metrics and selecting a proper benchmark. I would consider implement AIC and BIC for model evaluation if I know how. Also, since the pca tranformation reduce no dimension, part of the goal failed at that point. With no benefit reducing the features to be analyze, extra transformation causes extra loss of information.

More visualization techinique should be used instead of description by text.